In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from gensim import corpora, models
from pprint import pprint


In [2]:
year = [i for i in range(1957, 2022)]
sortedresult = pd.DataFrame()

In [3]:
dfWordList = pd.read_excel("./modi_data/nltk_word_cor.xlsx")
dfWordDel = dfWordList[dfWordList["correction"] == "delete"]
dfWordMod = dfWordList[dfWordList["correction"] != "delete"]

seriesDelete = dfWordDel["word"]
stopword = []
for word in seriesDelete.values:
    stopword.append(word)
#print(stopword)

seriesModify = dfWordMod["word"]
modiword = []
for word in seriesModify.values:
    modiword.append(word)
#print(len(modiword))

seriesModify2 = dfWordMod["correction"]
modiword2 = []
for word in seriesModify2.values:
    modiword2.append(word)
#print(len(modiword2))

In [4]:
stop_word = set(stopword + stopwords.words('english') + list(string.punctuation))
#print(stop_word)

In [5]:
def nltkTokenizer(raw, stopword=stop_word):
    list = []

    for word in word_tokenize(raw): #raw data
        #print(word)
        wl = word.lower()
        if wl not in stopword and len(word) > 1:
            if wl in modiword:
                wl = modiword2[modiword.index(wl)].lower()
            list.append(wl)            
    return list

In [6]:
for i in range(len(year)):
    print("==== "+str(year[i])+" ====")
    data = pd.read_csv("./modi_data/eng_"+str(year[i])+".csv")

    tokenized = data["eng_full"].apply(lambda row: nltkTokenizer(row))
    #print(tokenized)
    print("========= tokenization completed =========")

    id2word = corpora.Dictionary(tokenized)
    corpus=[id2word.doc2bow(text) for text in tokenized]
    print("# words in total : ", len(id2word))
    print("# documents : ", len(corpus))

    #tfidf
    print("==== calculating tfidf ====")
    tfidf = models.TfidfModel(corpus)

    #tfidf per doc
    tfidflist = []
    for doc in tfidf[corpus]:
        inner_list = [0]*len(id2word) 
        for id, freq in doc:
            inner_list[id] = np.around(freq, decimals=2) #put tfidf value in the place matching its index
        tfidflist.append(inner_list)
    #print(len(tfidflist))

    tfidf_df = pd.DataFrame(tfidflist)
    tfidf_df.columns = [id2word[i] for i in range(len(id2word))] #set columns' names as words
    #print(tfidf_df)

    total_df = pd.concat([data[["year", "no"]], tfidf_df], axis=1)
    #print(total_df)
    total_df.to_excel("./final_data/tfidf"+str(year[i])+"_nltk.xlsx") 

    #sum of tfidf for each word
    columnsum = pd.DataFrame(total_df.sum(axis=0)).T
    columnsum = columnsum.drop(['no'], axis=1)
    columnsum['year'] = year[i]
    #print(columnsum)
    columnsum.to_excel("./final_data/sum"+str(year[i])+"_nltk.xlsx")

    #sort tfidf value in descending order
    columnsum = columnsum.sort_values(by=0, axis=1, ascending=False)
    print(columnsum)
    columnsum.to_excel("./final_data/sorted"+str(year[i])+"_nltk.xlsx")

    print("==== completed ====")

nted  \
0  1988  1988    1.76    1.57  1.54     1.46      1.36      1.17      1.13   

   perceptions  ...  rather  practical  basis  conflict  variables  satisfied  \
0         1.08  ...    0.09       0.09   0.08      0.08       0.08       0.08   

   orientation  measure  male  expressed  
0         0.08     0.08  0.08       0.08  

[1 rows x 910 columns]
==== completed ====
==== 1989 ====
========= tokenization completed =========
# words in total :  831
# documents :  38
==== calculating tfidf ====
   year  year  book  review  gifted  teachers  students  parents  \
0  1989  1989  1.62    1.62    1.22      1.11      1.04     0.99   

   creative-thinking  program  ...  primary  precocity  penbow  paternal  \
0               0.99     0.96  ...     0.07       0.07    0.07      0.07   

   parental  mathematically  math/science  males  investigation  novice  
0      0.07            0.07          0.07   0.07           0.07    0.07  

[1 rows x 832 columns]
==== completed ====
==== 1990 

In [7]:
# tfidf for full data
data = pd.read_csv("./modi_data/eng_full.csv")

tokenized = data["eng_full"].apply(lambda row: nltkTokenizer(row))
#print(tokenized)
print("========= tokenization completed =========")

id2word = corpora.Dictionary(tokenized)
corpus=[id2word.doc2bow(text) for text in tokenized]
print("# words in total : ", len(id2word))
print("# documents : ", len(corpus))

#tfidf
print("==== calculating tfidf ====")
tfidf = models.TfidfModel(corpus)

#tfidf per doc
tfidflist = []
for doc in tfidf[corpus]:
    inner_list = [0]*len(id2word) 
    for id, freq in doc:
        inner_list[id] = np.around(freq, decimals=2) #put tfidf value in the place matching its index
    tfidflist.append(inner_list)
#print(len(tfidflist))

tfidf_df = pd.DataFrame(tfidflist)
tfidf_df.columns = [id2word[i] for i in range(len(id2word))] #set columns' names as words
#print(tfidf_df)

total_df = pd.concat([data[["year", "no"]], tfidf_df], axis=1)
#print(total_df)
total_df.to_excel("./final_data/tfidf_full_nltk.xlsx") 

#sum of tfidf for each word
columnsum = pd.DataFrame(total_df.sum(axis=0)).T
columnsum = columnsum.drop(['no'], axis=1)
columnsum['year'] = year[i]
#print(columnsum)
columnsum.to_excel("./final_data/sum_full_nltk.xlsx")

#sort tfidf value in descending order
columnsum = columnsum.sort_values(by=0, axis=1, ascending=False)
print(columnsum)
columnsum.to_excel("./final_data/sorted_full_nltk.xlsx")

print("==== completed ====")

========= tokenization completed =========
# words in total :  9699
# documents :  2537
==== calculating tfidf ====
   year  year  gifted  students  book  review  children  education  \
0  2021  2021   72.05     57.07  49.5   45.51     45.27      42.36   

   creativity  creative  ...  moderating  initiating  abstainers  fruitful  \
0       34.54     32.13  ...        0.04        0.04        0.04      0.04   

   experimenting  distancing  desirability  dearth  alarming  \
0           0.04        0.04          0.04    0.04      0.04   

   use/experimentation  
0                 0.04  

[1 rows x 9700 columns]
==== completed ====
